# Código para aplicar a krigagem nos dados de tipos de solo

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import gstools as gs
import geopandas as gpd
import rioxarray as rxr

from skgstat import Variogram
from tqdm import tqdm, trange
from pykrige.ok import OrdinaryKriging

In [2]:
dados    = pd.read_excel(r"D:\Mestrado\Trabalho Final\Dados\Levantamento em Campo\Compiled.xlsx", sheet_name="Analise Solo")
uso_solo = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/USOSOLO.tif")

In [3]:
dados.dropna(inplace=True, ignore_index=True)

points = gpd.points_from_xy(dados["lat"], dados["lon"], crs="EPSG:4326")
dados = gpd.GeoDataFrame(dados[["Ponto", "PROF", "Argila", "Silte", "Areia(Grossa)", "Areia(Fina)"]], geometry=points, crs="EPSG:4326")
dados.to_crs("EPSG:31983", inplace=True)

dados

,Ponto,PROF,Argila,Silte,Areia(Grossa),Areia(Fina),geometry
0,02,0 - 2,27.0,3.0,58.0,12.0,POINT (592224.603 7787265.87)
1,02,0 - 20,41.0,9.0,37.0,13.0,POINT (592224.603 7787265.87)
2,02,60 - 80,42.0,5.0,37.0,16.0,POINT (592224.603 7787265.87)
3,04,0 - 2,11.0,12.0,49.0,28.0,POINT (592368.605 7790837.259)
4,04,0 - 20,13.0,19.0,53.0,15.0,POINT (592368.605 7790837.259)
...,...,...,...,...,...,...,...
196,161,"1,4m",8.0,21.0,31.0,40.0,POINT (595970.168 7791192.033)
197,162,0 - 2,14.0,31.0,37.0,18.0,POINT (595908.053 7791452.29)
198,162,0 - 20,21.0,16.0,42.0,21.0,POINT (595908.053 7791452.29)
199,162,60 - 80,14.0,41.0,23.0,22.0,POINT (595908.053 7791452.29)


In [4]:
profs = dados["PROF"].unique()
profs

array(['0 - 2', '0 - 20', '60 - 80', '40 - 60', '20 - 40', '0  - 20',
       '0 - 10', '30 - 50', '50 - 70', '40 - 50', '0 - 15', '1,4m',
       '1,6m'], dtype=object)

In [5]:
prof_02 = dados[dados["PROF"]=="0 - 2"]
prof_20 = dados[(dados["PROF"]=="0 - 20")|(dados["PROF"]=="0 - 10")|(dados["PROF"]=="0  - 20")|(dados["PROF"]=="0 - 15")]
prof_80 = dados[dados["PROF"].isin(["60 - 80", "40 - 60", "20 - 40", "30 - 50", "50 - 70", "40 - 50"])]

len(prof_02), len(prof_20), len(prof_80)

(51, 80, 68)

### Aplicando a Krigagem

In [6]:
gridx = uso_solo.x.values
gridy = uso_solo.y.values

gridx.shape, gridy.shape

((9099,), (9099,))

In [ ]:
uso_solo = rxr.open_rasterio(r"D:/Mestrado/Trabalho Final/SIG/USOSOLO.tif")
gridx = uso_solo.x.values
gridy = uso_solo.y.values

# Para evitar estouro da memória
step = 500

tipos = ["Argila", "Silte", "Areia(Grossa)", "Areia(Fina)"]
profundidades = [2, 20, 80]

for prof in profundidades:
    if prof == 2:
        profs = prof_02
    elif prof == 20:
        profs = prof_20
    else:
        profs = prof_80
    
    x = profs.geometry.x
    y = profs.geometry.y
    coords = np.vstack([x, y]).T

    for tipo in tipos:
        z = profs[tipo].values

        V = Variogram(
            coords, z,
            model='spherical',     # pode testar: 'exponential', 'gaussian'
            maxlag='median',       # distância máxima considerada
            n_lags=8               # número de "bins" do semivariograma
        )

        rang   = V.parameters[0]
        sill   = V.parameters[1]
        nugget = V.parameters[2]

        model = gs.Spherical(dim=2, var=sill, len_scale=rang, nugget=nugget)
        ok = gs.krige.Ordinary(model, cond_pos=[x, y], cond_val=z)

        X, Y = np.meshgrid(gridx, gridy)
        print(f"Gerando {tipo} em {prof}cm")
        field, var = ok((X, Y), chunk_size=20000)   # field = valores preditos, var = variância

        print(f"Salvando {tipo} em {prof}cm")

        # Pega o template de coordenadas
        coords = {"y": uso_solo.y, "x": uso_solo.x}

        da_pred = xr.DataArray(
            field,
            coords=coords,
            dims=("y","x"),
            name="pred"
        )

        da_var = xr.DataArray(
            var,
            coords=coords,
            dims=("y","x"),
            name="var"
        )
        da_pred = da_pred.expand_dims("band").assign_coords(band=[1])
        da_var = da_var.expand_dims("band").assign_coords(band=[2])
        stack = xr.concat([da_pred, da_var], dim="band")

        stack = stack.rio.write_crs(uso_solo.rio.crs)
        stack = stack.rio.write_transform(uso_solo.rio.transform())
        stack.rio.to_raster(fr"D:/Mestrado/Trabalho Final/SIG/{tipo}_{prof}.tif", compress="LZW")

        break
    break

Gerando Argila em 2cm
